In [60]:
orthophotos[4][-26:]

'35-2006-0305-6780-LA93.tif'

In [136]:
config = {
    "mnhc_path":"données/MNHC/",
    "ortho_path":"données/orthophotos/2006_OK/rvb/",
    "irc_path":"données/orthophotos/2006_OK/rvb/",
    'rgb_vignettes_path':'vignettes/rgb/',
    'mask_vignettes_path':'vignettes/mask/',
    'rgb_coordinates_pos':-17,
    'rgb_name_pos':-26
}

path_to_orthophoto_rgb = config["ortho_path"] + orthophotos[4][config['rgb_name_pos']:]
path_to_orthophoto_irc = config["irc_path"] + orthophotos[4][config['rgb_name_pos']:] # A Changer plus tard
mnhc_tiff = config['mnhc_path'] + "Diff_MNS_CORREL_1-0_LAMB93_20FD3525_" + str(orthophotos[4][-17:-9]).replace('-', '_') + ".tif"
# position
dept = os.path.basename(path_to_orthophoto_rgb)[:2]
rgb_x = path_to_orthophoto_rgb[config['rgb_coordinates_pos']: config['rgb_coordinates_pos']+3]
rgb_y = path_to_orthophoto_rgb[config['rgb_coordinates_pos']+4: config['rgb_coordinates_pos']+8]
mnhc_dir_dept = config['mnhc_path']+dept+'_2020/'

In [137]:
config["irc_path"] + orthophotos[4][config['rgb_name_pos']:] 

'données/orthophotos/2006_OK/rvb/35-2006-0305-6780-LA93.tif'

In [138]:
# images IRC et RGB
# images IRC et RGB
ortho_irc_file = config["irc_path"] + '/' + dept + '-2020-0'+rgb_x+'-'+rgb_y+'-LA93-0M20-IRC-E080.jp2'
# ouverture
ortho_rgb = cv2.resize(np.asarray(Image.open(path_to_orthophoto_rgb)), (10000, 10000), interpolation=cv2.INTER_AREA)
ortho_irc = cv2.resize(np.asarray(Image.open(path_to_orthophoto_irc)), (10000, 10000), interpolation=cv2.INTER_AREA)
ndvi = np.divide(ortho_irc[:,:,0]-ortho_irc[:,:,1],ortho_irc[:,:,0]+ortho_irc[:,:,1], where =  (ortho_irc[:,:,0]+ortho_irc[:,:,1])!=0 )
ortho_irc = None    

In [139]:
if not os.path.exists(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/'):
    os.makedirs(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/')
if not os.path.exists(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/'):
    os.makedirs(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/')

i = 0
# vignette_data = []
for mnhc_x in tqdm(range(0,5)):
    for mnhc_y in range(0,5):
        i+=1
        
        print(f'pic {i}: {mnhc_x}, {mnhc_y}')
        mnhc_file = mnhc_dir_dept+'Diff_MNS_CORREL_1-0_LAMB93_20FD'+dept+'25_'+str(int(int(rgb_x)+mnhc_x))+'_'+str(int(int(rgb_y)-mnhc_y))+'.tif'
        
        if os.path.exists(mnhc_file):
            print(mnhc_file)
            mnhc = np.asarray(Image.open(mnhc_file))
            w, h = mnhc.shape
            mask = np.logical_and(np.logical_and(mnhc > 3., 
                                                 np.logical_or(mnhc > 5., 
                                                               std_convoluted(mnhc,N=10) > .5)
                                                ),
                                  ndvi[mnhc_x*w:(mnhc_x+1)*w, mnhc_y*h:(mnhc_y+1)*h] > 0) \
            .astype('uint8')

            for x in range(0, w-IMAGE_SIZE+1, IMAGE_SIZE):
                for y in range(0, h-IMAGE_SIZE+1, IMAGE_SIZE):
                    # print(x, y)
                    crop_rgb = ortho_rgb[mnhc_y*h+y: mnhc_y*h+y+IMAGE_SIZE, mnhc_x*w+x: mnhc_x*w+x+IMAGE_SIZE, :]
                    crop_mask = mask[y:y+IMAGE_SIZE, x:x+IMAGE_SIZE]
                    # print(crop_mask.dtype, crop_mask.shape)
                    # filtre rgb
                    # enregistrement
                    pos = dept+'_'+str(int((int(rgb_x)+mnhc_x)*1000+x/2))+'_'+str(int((int(rgb_y)-mnhc_y)*1000+y/2))
                    
                    cv2.imwrite(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/' + 'rgb_' + str(pos) + '.jpg', crop_rgb)
                    cv2.imwrite(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/' + 'mask_'+ str(pos) + '.png', crop_mask*255)
                    
                    # caractéristiques image RGB : moyenne/ecart-type hors et dans masque, surface masque
                    # Does not work
                    # mask = np.stack((crop_mask,)*3, axis=-1)
                    # non_mask = np.logical_not(mask)
                    # res = [pos,crop_mask.sum()] + \
                    #        np.mean(crop_rgb, axis=(0,1), where=mask).tolist() + \
                    #        np.mean(crop_rgb, axis=(0,1), where=non_mask).tolist() + \
                    #        np.std(crop_rgb, axis=(0,1), where=mask).tolist() + \
                    #        np.std(crop_rgb, axis=(0,1), where=non_mask).tolist()
                    # vignette_data.append(res)
            break
                    
# df = pd.DataFrame(vignette_data,columns = ('file','wood_size','w_mean_R','w_mean_G','w_mean_B','nw_mean_R','nw_mean_G','nw_mean_B','w_std_R','w_std_G','w_std_B','nw_std_R','nw_std_G','nw_std_B'))

  0%|                 | 0/5 [00:00<?, ?it/s]

pic 1: 0, 0
données/MNHC/35_2020/Diff_MNS_CORREL_1-0_LAMB93_20FD3525_305_6780.tif


 20%|█▊       | 1/5 [00:08<00:34,  8.50s/it]

pic 2: 1, 0
données/MNHC/35_2020/Diff_MNS_CORREL_1-0_LAMB93_20FD3525_306_6780.tif


 40%|███▌     | 2/5 [00:15<00:23,  7.67s/it]

...

 60%|█████▍   | 3/5 [00:23<00:15,  7.70s/it]

pic 4: 3, 0
données/MNHC/35_2020/Diff_MNS_CORREL_1-0_LAMB93_20FD3525_308_6780.tif


 80%|███████▏ | 4/5 [00:30<00:07,  7.67s/it]

pic 5: 4, 0
données/MNHC/35_2020/Diff_MNS_CORREL_1-0_LAMB93_20FD3525_309_6780.tif


100%|█████████| 5/5 [00:38<00:00,  7.70s/it]


In [116]:
config['rgb_vignettes_path'] + 'rgb_' + str(pos) + '.jpg'

'vignettes/rgb/rgb_35_309768.0_6780768.0.jpg'

In [112]:
dept+'_'+str((int(rgb_x)+mnhc_x)*1000+x/2)+'_'+str((int(rgb_y)-mnhc_y)*1000+y/2)

'35_309768.0_6780768.0'

In [127]:
256*8

2048

In [98]:
np.mean(crop_rgb, axis=(0,1))

array([74.40789795, 90.95449829, 64.05075073])

In [ ]:
from PIL import Image
from IPython.display import display
import glob,os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from skimage.transform import resize, downscale_local_mean
import scipy
import cv2
from tqdm import tqdm

In [ ]:
config = {
    "mnhc_path":"données/MNHC/",
    "ortho_path":"données/BDORTHO/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00121/OHR_RVB_0M20_JP2-E080_LAMB93_D35-2020/",# "données/orthophotos/2006_OK/rvb/",
    "irc_path":"données/BDORTHO_IRC/ORTHOHR_1-0_IRC-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR_1-0_IRC-0M20_JP2-E080_LAMB93_D035_2020-01-01/ORTHOHR/1_DONNEES_LIVRAISON_2021-06-00110/OHR_IRC_0M20_JP2-E080_LAMB93_D35-2020/",# "données/orthophotos/2006_OK/rvb/",
    'rgb_vignettes_path':'vignettes/rgb/',
    'mask_vignettes_path':'vignettes/mask/',
    'rgb_coordinates_pos':-27,
    'rgb_name_pos':-36,
    'img_size':256
}

In [ ]:
path_to_orthophoto_rgb = config['ortho_path'] + '35-2020-0305-6780-LA93-0M20-E080.jp2'

In [ ]:
config["irc_path"] + path_to_orthophoto_rgb[config['rgb_name_pos']:]

In [ ]:
path_to_orthophoto_rgb[config['rgb_coordinates_pos']+4: config['rgb_coordinates_pos']+8]

In [ ]:
def std_convoluted(im, N):
    """
    Calculate the standard deviation using a convolution operation.

    Parameters:
    - im (numpy.ndarray): Input image as a NumPy array.
    - N (int): Radius of the convolution kernel.

    Returns:
    - numpy.ndarray: Standard deviation image.
    """
    # Calculate squared image and initialize kernel
    im2 = im**2
    ones = np.ones(im.shape)
    kernel = np.ones((2*N+1, 2*N+1))

    # Convolve the image and squared image with the kernel
    s = scipy.signal.convolve2d(im, kernel, mode="same")
    s2 = scipy.signal.convolve2d(im2, kernel, mode="same")
    ns = scipy.signal.convolve2d(ones, kernel, mode="same")

    # Calculate the standard deviation image
    return np.sqrt((s2 - s**2 / ns) / ns)

In [ ]:
def my_func(path_to_orthophoto_rgb):
    
    path_to_orthophoto_irc = config["irc_path"] + path_to_orthophoto_rgb[config['rgb_name_pos']:] # A Changer plus tard
    mnhc_tiff = config['mnhc_path'] + "Diff_MNS_CORREL_1-0_LAMB93_20FD3525_" + str(path_to_orthophoto_rgb[-17:-9]).replace('-', '_') + ".tif"
    
    # position
    dept = os.path.basename(path_to_orthophoto_rgb)[:2]
    rgb_x = path_to_orthophoto_rgb[config['rgb_coordinates_pos']: config['rgb_coordinates_pos']+3]
    rgb_y = path_to_orthophoto_rgb[config['rgb_coordinates_pos']+4: config['rgb_coordinates_pos']+8]
    mnhc_dir_dept = config['mnhc_path']+dept+'_2020/'
    
    # images IRC et RGB
    # ortho_irc_file = config["irc_path"] + '/' + dept + '-2020-0'+rgb_x+'-'+rgb_y+'-LA93-0M20-IRC-E080.jp2'
    # ouverture
    ortho_rgb = cv2.resize(np.asarray(Image.open(path_to_orthophoto_rgb)), (10000, 10000), interpolation=cv2.INTER_AREA)
    ortho_irc = cv2.resize(np.asarray(Image.open(path_to_orthophoto_irc)), (10000, 10000), interpolation=cv2.INTER_AREA)
    ndvi = np.divide(ortho_irc[:,:,0]-ortho_irc[:,:,1],ortho_irc[:,:,0]+ortho_irc[:,:,1], where =  (ortho_irc[:,:,0]+ortho_irc[:,:,1])!=0 )
    ortho_irc = None 
    
    if not os.path.exists(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/'):
        os.makedirs(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/')
    if not os.path.exists(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/'):
        os.makedirs(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/')
    
    # vignette_data = []
    for mnhc_x in tqdm(range(0,5)):
        for mnhc_y in range(0,5):
            
            mnhc_file = mnhc_dir_dept+'Diff_MNS_CORREL_1-0_LAMB93_20FD'+dept+'25_'+str(int(int(rgb_x)+mnhc_x))+'_'+str(int(int(rgb_y)-mnhc_y))+'.tif'
            
            if os.path.exists(mnhc_file):
                print(mnhc_file)
                mnhc = np.asarray(Image.open(mnhc_file))
                w, h = mnhc.shape
                mask = np.logical_and(np.logical_and(mnhc > 3., 
                                                     np.logical_or(mnhc > 5., 
                                                                   std_convoluted(mnhc,N=10) > .5)
                                                    ),
                                      ndvi[mnhc_x*w:(mnhc_x+1)*w, mnhc_y*h:(mnhc_y+1)*h] > 0) \
                .astype('uint8')
                
                for x in range(0, w-config['img_size']+1, config['img_size']):
                    for y in range(0, h-config['img_size']+1, config['img_size']):
                        # print(x, y)
                        crop_rgb = ortho_rgb[mnhc_y*h+y: mnhc_y*h+y+config['img_size'], mnhc_x*w+x: mnhc_x*w+x+config['img_size'], :]
                        crop_mask = mask[y:y+config['img_size'], x:x+config['img_size']]
                        # print(crop_mask.dtype, crop_mask.shape)
                        # filtre rgb
                        # enregistrement
                        pos = dept+'_'+str(int((int(rgb_x)+mnhc_x)*1000+x/2))+'_'+str(int((int(rgb_y)-mnhc_y)*1000+y/2))
                        
                        cv2.imwrite(config['rgb_vignettes_path'] + f'rgb_{rgb_x}_{rgb_y}/' + 'rgb_' + str(pos) + '.jpg', crop_rgb)
                        cv2.imwrite(config['mask_vignettes_path'] + f'mask_{rgb_x}_{rgb_y}/' + 'mask_'+ str(pos) + '.png', crop_mask*255)
                
                # Si besoin de plus d'images, commenter le break
                break

In [ ]:
path_to_orthophoto_rgb = 'données/orthophotos/2006_OK/rvb/35-2006-0310-6780-LA93.tif'

my_func(path_to_orthophoto_rgb)